In [1]:
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import os 

In [2]:
# Laden des vortrainierten VGG16-Modells
model = VGG16(weights='imagenet', include_top=False)

# Datenpfad zum Ordner mit den Bildern

data_path = 'C:/Users/Lorena/Documents/Studium/Masterarbeit/Ausarbeitung_files/Final/Data/Images_noDuplicates_labled/'

# Liste für Bilder und Labels
images = []
labels = []

In [3]:
# Schleife über die Bilder im Ordner
for img_name in os.listdir(data_path):
    img = cv2.imread(os.path.join(data_path, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV liest Bilder im BGR-Format ein, daher Umwandlung in RGB
    img = cv2.resize(img, (224, 224))  # VGG16 akzeptiert Bilder in der Größe 224x224
    images.append(img)
    labels.append(img_name.split('.')[0])  # Annahme: Die Bilddateien haben Namen wie 'label1.jpg', 'label2.jpg', usw.

# Konvertierung der Bilder und Labels in Numpy-Arrays
images = np.array(images)
labels = np.array(labels)

In [4]:
# Extraktion der Features für jedes Bild
features = []
for img in images:
    img = preprocess_input(img)
    feature = model.predict(np.expand_dims(img, axis=0))
    features.append(feature.flatten())

In [5]:
# Dimensionsreduktion der Features
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features)

# Klassifikation mit k-NN
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(features_pca, labels)

# Vorhersage auf dem reduzierten Feature-Raum
y_pred = knn.predict(features_pca)

C:\tools\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [21]:
color_mapping = {
    'akiec': 'AKIEC',
    'bcc': 'BCC',
    'mel': 'MEL',
    'vasc': 'VASC',
    'bkl': 'BKL', 
    'df': 'DF', 
    'nv': 'NV'
}

# Liste für die zugeordneten Farben
colors = []

# Labels durchgehen und die entsprechende Farbe zuordnen
for label in labels:
    for key in color_mapping:
        if key in label:
            colors.append(color_mapping[key])
            break

In [29]:
import plotly.express as px
import pandas as pd

# Create a DataFrame with the features_pca and colors
data = {'Feature 1': features_pca[:, 0],
        'Feature 2': features_pca[:, 1],
        'Colors': colors}
df = pd.DataFrame(data)

# Define a custom color scale
color_scale = ['#BF364F', "#3A3684", '#D92B2B', '#D95323', '#F28F16', '#F2CB05', '#5A3473']


fig = px.scatter(df, x='Feature 1', y='Feature 2', color='Colors', title='Identifizierte Klassen', color_discrete_sequence=color_scale)
#fig.update_traces(marker=dict(opacity=0.2))  # Set the marker opacity to 0.5 (half-transparent)

# Plot the scatter plot using Plotly
#fig = px.scatter(df, x='Feature 1', y='Feature 2', color='Colors', title='Identifizierte Klassen')
# Customize the colorbar title
fig.update_coloraxes(colorbar_title='Custom Colorbar Title')

# Customize the layout
fig.update_layout(
    plot_bgcolor='white',   # Set the plot background color
    paper_bgcolor='white'   # Set the paper (around the plot) background color
)

fig.show()

In [30]:
image_path = "C:/Users/Lorena/Documents/Studium/Masterarbeit/Ausarbeitung_Bilder/"
# TODO: Uncomment fig.write_image
fig.write_image(image_path + "transferLearning_VGG16.pdf")